### Importing Libraries

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import math
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import missingno
from statsmodels.tsa.stattools import acf, pacf
import yaml
import os
from pathlib import Path

### Import modules

In [ ]:
import sys
sys.path.append("../src")

import data_exploration as de
import model_training as mt

### Load necessary directories

In [ ]:
current_dir = Path(os.getcwd())
root_dir = current_dir
while 'Portfolio Optimization using SPO' in root_dir.parts:
    root_dir = root_dir.parent
    if root_dir == Path(root_dir.root):
        print("Root directory not found.")
        break

In [ ]:
config_path = root_dir / "Portfolio Optimization using SPO" / "config" / "config.yml"
data_path = root_dir / "Portfolio Optimization using SPO" / "data" / "dat_518_companies.csv"

In [ ]:
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

### Import data

In [ ]:
df_original = pd.read_csv(data_path)
df_original.head()

In [ ]:
df_original.describe()

In [ ]:
print(f'Data has {len(df_original.ticker.unique())} unique securities')
print(f'Data has {len(df_original.sector.unique())} unique sectors')
print(f'Data has {len(df_original.industry.unique())} unique industries')

### Check for missing data

In [ ]:
missingno.matrix(df_original,figsize=(10,10), fontsize=12)

We can see sentiment has lot of missing values

### Check the consistency of data for each security

In [ ]:
fig = px.line(df_original, x="date", y="return_t_plus_1", color='ticker')
fig.show()

Not all the securities have same staring and ending date, so select a target ticker and other 5 securities having same start and end date. The securities "AAPL", "ABMD", "ADBE", "ADI", "ADP" has same start and end date, and belong to different sector-industry combination.

### Segregate target ticker for EDA

In [ ]:
# Select 5 tickers having same number of datapoints with same start and end date
df_ticker_target = df_original[df_original['ticker'] == config['target_ticker']]

# Drop default index
df_ticker_target.drop(['Unnamed: 0'], axis=1 , inplace = True)

# Reset index
df_ticker_target.reset_index(inplace = True)

In [ ]:
missingno.matrix(df_ticker_target,figsize=(10,10), fontsize=12)

Only sentiment has missing values so calculate the percentage of missing values from that column

In [ ]:
percent_missing = df_ticker_target["sentiment"].isna().sum()/1043
print(f'The sentiment has {percent_missing * 100}% values missing')

So dropping it and also dropping categorical features is the next step

### Select numerical feature and features with no missing data

In [ ]:
df_ticker_target_numerical_features = df_ticker_target[config['numerical_features']]

In [ ]:
# save for future use
AAPL_df_path = root_dir / "Portfolio Optimization using SPO" / "data" / "AAPL_df.csv"    
df_ticker_target_numerical_features.to_csv(AAPL_df_path, index = False)

In [ ]:
de.plot_corr_plots(df_ticker_target_numerical_features)

We can see the many features are correlated with each other lets see how many features have correlation more than 70%

### Identify highly correlated features

In [ ]:
correlated_feats = de.identify_correlated(df_ticker_target_numerical_features, threshold=0.7)
correlated_feats

These features have more than 70% correlation

### Plot distribution for each feature

In [ ]:
numerical_cols = df_ticker_target_numerical_features.columns

for column in numerical_cols:
    de.plot_histograms(df_ticker_AAPL_numerical_features, column)

## Pre-processing the dataset

### Creating cost matrix

In [ ]:
# Select 5 tickers having same number of datapoints with same start and end date
df_ticker_0 = df_original[df_original['ticker'] == config["other_securities"][0]]
df_ticker_1 = df_original[df_original['ticker'] == config["other_securities"][1]]
df_ticker_2 = df_original[df_original['ticker'] == config["other_securities"][2]]
df_ticker_3 = df_original[df_original['ticker'] == config["other_securities"][3]]

# Drop default index
df_ticker_0.drop(['Unnamed: 0'], axis=1 , inplace = True)
df_ticker_1.drop(['Unnamed: 0'], axis=1 , inplace = True)
df_ticker_2.drop(['Unnamed: 0'], axis=1 , inplace = True)
df_ticker_3.drop(['Unnamed: 0'], axis=1 , inplace = True)

# Reset index
df_ticker_target.reset_index(inplace = True)
df_ticker_0.reset_index(inplace = True)
df_ticker_1.reset_index(inplace = True)
df_ticker_2.reset_index(inplace = True)
df_ticker_3.reset_index(inplace = True)

# Create a new dataframe with t+1 returns of above 5 tickers
df_final_returns = pd.DataFrame()
df_final_returns[config["target_ticker"]] = df_ticker_target.return_t_plus_1
df_final_returns[config["other_securities"][0]] = df_ticker_0.return_t_plus_1
df_final_returns[config["other_securities"][1]] = df_ticker_1.return_t_plus_1
df_final_returns[config["other_securities"][2]] = df_ticker_2.return_t_plus_1
df_final_returns[config["other_securities"][3]] = df_ticker_3.return_t_plus_1

# Preview the dataframe
df_final_returns.head()

In [ ]:
# save for future use
cost_mat_path = root_dir / "Portfolio Optimization using SPO" / "data" / "cost_mat.csv"    
df_final_returns.to_csv(cost_mat_path, index = False)

### Create covariance matrix

In [ ]:
# covatiance matrix calculation
df_returns_transpose = np.transpose(df_final_returns)
sigma = np.cov(df_returns_transpose)
sigma_df = pd.DataFrame(sigma)
sigma_df

In [ ]:
# save for future use
sigma_path = root_dir / "Portfolio Optimization using SPO" / "data" / "sigma_df.csv"    
sigma_df.to_csv(sigma_path, index = False)

### Calculate $\gamma$ (risk factor)

In [ ]:
# risk factor calculation
avg_vec = np.ones(5)/5
gamma = np.transpose(avg_vec)@sigma@avg_vec
gamma = 2.25*gamma
print(gamma)

Note: write this gamma in the config file if not already there otherwise the optimization will be calculated with wrong gamma. Or there will be an error.